In [1]:
%time
import numpy as np
import cv2 as cv
import os
import faiss

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 7.87 µs


In [2]:
def get_sift_keypoints(img):
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create(nfeatures=5000)
    kp, des = sift.detectAndCompute(gray, None)
    return kp, des

In [3]:
labels = []
features = []
for image_name in os.listdir('images'):
    building_name = image_name[0:image_name.find('.')]
    
    image_path = 'images/' + image_name
    img = cv.imread(image_path)
    
    kp, des = get_sift_keypoints(img)
    labels += [building_name for i in range(len(kp))]
    
    features.append(np.vstack(des))
    
    # img=cv.drawKeypoints(gray,kp,img)
    # cv.imshow('image',img)
    # cv.waitKey(0)

In [4]:
len(labels)

112722

In [5]:
features_ar = np.vstack(features)
features_ar.shape

(112722, 128)

In [6]:
index = faiss.IndexFlatL2(128)
index.add(features_ar)
print(index.ntotal)

112722


In [7]:
def find_closest_image_match(img):
    return None

In [8]:
search_image_path = 'search.jpg'
img = cv.imread(search_image_path)
kp, des = get_sift_keypoints(img)
k = 1

In [9]:
preds = {}
for d in des:
    D, I = index.search(d.reshape((1, 128)), k)
    pred = labels[I[0, 0]]
    if pred not in preds:
        preds[pred] = 0
    preds[pred] += 1

preds

{'Norris_University_Center_3': 8,
 'Walter_Annenberg_Hall_1': 10,
 'Fisk_Hall_4': 20,
 'Swift_Hall_2': 5,
 'The_Rock_1': 41,
 'Harris_Hall_2': 30,
 'The_Virginia_Wadsworth_Wirtz_Center_3': 83,
 'The_Rock_4': 16,
 'Kellogg_Global_Hub_4': 25,
 'Pick_Staiger_Concert_Hall_1': 93,
 'Bienen_School_of_Music_4': 45,
 'Donald_P_Jacobs_Center_2': 3,
 'Bienen_School_of_Music_2': 69,
 'The_Virginia_Wadsworth_Wirtz_Center_1': 80,
 'Garrett-Evangelical_Theological_Seminary_1': 35,
 'kellogg': 148,
 'Norris_University_Center_2': 110,
 'kellogg_full': 222,
 'Mudd_Library_3': 11,
 'The_Virginia_Wadsworth_Wirtz_Center_4': 30,
 'Bienen_School_of_Music_1': 68,
 'Mudd_Library_1': 10,
 'University_Hall_2': 10,
 'Pick_Staiger_Concert_Hall_2': 140,
 'Ford_Motor_Company_Engineering_Design_Center_2': 16,
 'Ford_Motor_Company_Engineering_Design_Center_1': 12,
 'Deering_Library_2': 7,
 'University_Hall_3': 11,
 'Kellogg_Global_Hub_3': 12,
 'Crowe_Hall_1': 10,
 'Segal_Visitors_Center_4': 24,
 'University_Library_4